In [1]:
import pandas as pd
import sqlite3

In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
pd.read_sql('PRAGMA table_info(test);', connect)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [4]:
pd.read_sql('SELECT * FROM test LIMIT 10;', connect, index_col='uid')

,labname,first_commit_ts,first_view_ts
uid,,,
user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714


In [5]:
pd.read_sql('PRAGMA table_info(deadlines);', connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [6]:
pd.read_sql('SELECT * FROM deadlines LIMIT 10;', connect, index_col='index')

,labs,deadlines
index,,
0,laba04,1587945599
1,laba04s,1587945599
2,laba05,1588550399
4,laba06,1590364799
5,laba06s,1590364799
3,project1,1589673599


In [7]:
test_results = pd.read_sql("""SELECT 'before' AS time,
             AVG(CAST((julianday(test.first_commit_ts) -
             julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
             FROM test
             INNER JOIN  deadlines
             ON test.labname != 'project1' AND test.labname = deadlines.labs
             WHERE julianday(test.first_commit_ts) < julianday(test.first_view_ts)
             UNION
             SELECT 'after' AS time, AVG(CAST((julianday(test.first_commit_ts) -
             julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
             FROM test
             INNER JOIN  deadlines
             ON test.labname != 'project1' AND test.labname = deadlines.labs
             WHERE julianday(test.first_commit_ts) > julianday(test.first_view_ts)""",
             connect)
test_results

,time,avg_diff
0,after,-103.40625
1,before,-60.56250


In [8]:
control_results = pd.read_sql("""SELECT 'before' AS time, AVG(CAST((julianday(control.first_commit_ts) -
             julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
             FROM control
             INNER JOIN  deadlines
             ON control.labname != 'project1' AND control.labname = deadlines.labs
             WHERE julianday(control.first_commit_ts) < julianday(control.first_view_ts)
             UNION
             SELECT 'after' AS time, AVG(CAST((julianday(control.first_commit_ts) -
             julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
             FROM control
             INNER JOIN  deadlines
             ON control.labname != 'project1' AND control.labname = deadlines.labs
             WHERE julianday(control.first_commit_ts) > julianday(control.first_view_ts)""",
             connect)
control_results

,time,avg_diff
0,after,-112.710526
1,before,-99.464286
